### Steine auf Spielbrett verschieben
Zwei Spieler k&ouml;nnen abwechselnd auf einem Schachbrett Steine ihrer Farbe auf ein freies Feld verschieben. Die Klasse `Game` hat folgende (Daten)-Attribute und Methoden:
- `position`: Dictionary mit Schl&uuml;ssel 0 oder 1 (Spieler). Die Werte sind Mengen mit Feldern (Zahlen von 0-63).
- `ptm`: Player to move (0 oder 1)
- `move(player, src, target)`: verschiebe einen Stein von Spieler `player` vom Feld `src` auf das Feld `target`
- `is_legal(player, src, target)`: testet, ob Zug legal  

Nachdem die Methode `move` die Spielposition aktualisiert hat, ruft sie ein noch zu registrierendes Callback
auf, mit den Argumenten `'move_stone'` und `(player, src, target)`.  
Sp&auml;ter wird diese Callback eine Methode der `View` sein, welche den Spielzustand auf der Leinwand
aktualisiert.


In [2]:
class Game:
    
    def __init__(self):
        self.player_name = {0: 'Rot', 1: 'Gelb'}
        self.position = {0: set((0, 2, 4, 6)),
                         1: set((59,)),
                        }
        self.ptm = 0 # player to move
        
    def is_legal(self, player, src, target):
        # Spieler ist nicht am Zug
        if player != self.ptm:
            return False
        # Spieler hat keinen Stein auf Feld src
        if src not in self.position[player]:
            return False
        # Zielfeld kein legales Feld
        if target < 0 or target > 63:
            return False
        # Zielfeld besetzt
        if target in self.position[player]:
            return False
        if target in self.position[1-player]:
            return False
        
        return True
    
    def move(self, player, src, target):  
        if not self.is_legal(player, src, target):
            return
        
        # Index src aus der Menge entfernen und target hinzufuegen
        self.position[player].remove(src)
        self.position[player].add(target)
        
        # ruft noch zu registrierendes Callback auf
        self.callback('move_stone', (player, src, target))
        
        self.ptm = 1 - self.ptm
        
    def __repr__(self):
        return 'Am Zug: {}\nPosition: {}'.format(self.player_name[self.ptm], self.position)        

In [3]:
# Klasse Game testen
game = Game()
game.callback = lambda e, d: print('Message from Callback:',  e, d)
game

Am Zug: Rot
Position: {0: {0, 2, 4, 6}, 1: {59}}

In [4]:
game.move(0, 0, 9)
game

Message from Callback: move_stone (0, 0, 9)


Am Zug: Gelb
Position: {0: {2, 4, 6, 9}, 1: {59}}

In [5]:
# illegaler Zug, nichts passiert
game.move(0, 9, 10)
game

Am Zug: Gelb
Position: {0: {2, 4, 6, 9}, 1: {59}}

In [6]:
game.move(1, 59, 52)
game

Message from Callback: move_stone (1, 59, 52)


Am Zug: Rot
Position: {0: {2, 4, 6, 9}, 1: {52}}

### M&ouml;gliche Erweiterung: Lasse Computer Zug ausf&uuml;hren
Ist der Spielername eine Funktion, so wird diese Funktion mit den Argumenten 
`game.position` und `game.ptm` aufgerufen. Diese Funktion soll einen legalen Zug zur&uuml;ck geben. Im Moment gegeben wir einen zuf&auml;lligen Zug zur&uuml;ck.  

Wenn wir sp&auml;ter den Zug auf der Leinwand darstellen, soll sich der Computer mit der Antwort eine Sekunde Zeit lassen.
Dazu verwenden wir die Methode `time.sleep(1)` aus dem Modul `time`.  

Um unseren Code sp&auml;ter wieder verwenden zu k&ouml;nnen, speichern wir ihn im File
`game_steine.py` ab (Modul).

In [7]:
import random
def engine(state, ptm):
    # nicht besetzte Spielfelder
    options = [i for i in range(64) if i not in state[0] and i not in state[1]]
    return random.choice(list(state[ptm])), random.choice(options)

In [8]:
game

Am Zug: Rot
Position: {0: {2, 4, 6, 9}, 1: {52}}

In [9]:
# Engine testen
engine(game.position, 0)

(2, 23)

In [10]:
#%%file game_steine.py
import time, random

def engine(state, ptm):
    # nicht besetzte Spielfelder
    options = [i for i in range(64) if i not in state[0] and i not in state[1]]
    return random.choice(list(state[ptm])), random.choice(options)

class Game:
    
    def __init__(self):
        self.player_name = {0:'Rot', 1: 'Gelb'}
        self.position = {0: set((0, 2, 4, 6)),
                         1: set((59,)),
                        }
        self.ptm = 0 # player to move
        
    def is_legal(self, player, src, target):
        # Spieler ist nicht am Zug
        if player != self.ptm:
            return False
        # Spieler hat keinen Stein auf Feld src
        if src not in self.position[player]:
            return False
        # Zielfeld kein legales Feld
        if target < 0 or target > 63:
            return False
        # Zielfeld besetzt
        if target in self.position[player]:
            return False
        if target in self.position[1-player]:
            return False
        
        return True
    
    def move(self, player, src, target):  
        if not self.is_legal(player, src, target):
            return
        
        self.position[player].remove(src)
        self.position[player].add(target)
        
        # ruft noch zu registrierendes Callback auf
        self.callback('move_stone', (player, src, target))
        
        self.ptm = 1 - self.ptm
        
        player = self.player_name[self.ptm]
        if callable(player):
            src, target = player(self.position, self.ptm)
            time.sleep(1)
            self.move(self.ptm, src, target)
    def __repr__(self):
        return 'Am Zug: {}\nPosition: {}'.format(self.player_name[self.ptm], self.position)                

In [11]:
game = Game()
game.callback = lambda e, d: print('Message from Callback:',  e, d)
game.player_name[1] = engine
game

Am Zug: Rot
Position: {0: {0, 2, 4, 6}, 1: {59}}

In [12]:
game.move(0,0,10)
game

Message from Callback: move_stone (0, 0, 10)
Message from Callback: move_stone (1, 59, 29)


Am Zug: Rot
Position: {0: {2, 4, 6, 10}, 1: {29}}